In [ ]:
from monai.transforms import (
    EnsureChannelFirstd,
    Compose,
    LoadImaged,
    Orientationd,
    ScaleIntensityRanged,
    Resized,
    RandSpatialCropd,
    CenterSpatialCropd,
    RandScaleIntensityd,
    RandShiftIntensityd,
    RandGaussianSmoothd,
    RandRotated,
    ToTensord,
    MapTransform,
    Transform,
    EnsureTyped,
    ToDeviced,
    Transposed
)

import argparse
import sys
import os

current_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.dirname(current_dir)

sys.path.append(project_root)

from utils.util import *
sys.argv = ['run.py']


parser = argparse.ArgumentParser()

class OptimizedWindowLevelWidthd(MapTransform):
    def __init__(self, keys, window_level, window_width):
        super().__init__(keys)
        self.window_level = window_level
        self.window_width = window_width
        # 预先计算 lower 和 upper
        self.lower = self.window_level - self.window_width / 2
        self.upper = self.window_level + self.window_width / 2

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            # 使用 inplace 操作
            d[key] = (d[key] - self.lower) / (self.upper - self.lower)
            d[key].clamp_(0, 1)  # inplace clamp
        return d

# script for preprocessing ours

In [ ]:
from tqdm import tqdm
from PIL import Image

args = parser.parse_args()
args.resize_3d = 140
args.resize_slices = 40

datasets = [
    "RenalCLIP1.0", 
    "RenalCLIP2.0", 
    "RenalCLIP3.0", 
    "RenalCLIP3.1"
]

ori_root_template = "/cpfs01/projects-HDD/cfff-bb5d866c17c2_HDD/public/{dataset}/crop_img-KC_v3"
save_root = "/cpfs01/projects-SSD/cfff-bb5d866c17c2_SSD/public/RenalCLIP/nii_npy_aligned_preprocessed"
save_path = os.path.join(save_root, "one_kidney_kc_v3")


deterministic_preprocess_transform_list = [
    LoadImaged(keys=["image"], reader='numpyreader'),
    EnsureChannelFirstd(keys=["image"]),
    Orientationd(keys=["image"], axcodes="RAS"),
    Resized(keys="image", spatial_size=[args.resize_3d, args.resize_3d, args.resize_slices]),
    OptimizedWindowLevelWidthd(keys=["image"], window_level=50, window_width=500),
]
deterministic_transform = Compose(deterministic_preprocess_transform_list)

MODALITIES = ["N", "A", "V", "D"]

for dataset in datasets:
    ori_root = ori_root_template.format(dataset=dataset)

    if not os.path.exists(ori_root):
        print(f"Skipping {dataset} as ori root path does not exist.")
        continue

    patient_list = os.listdir(ori_root)
    for pid in tqdm(patient_list):
        try:
            if not os.path.exists(os.path.join(save_path, pid)):
                os.makedirs(os.path.join(save_path, pid))
            
            ct_list = os.listdir(os.path.join(ori_root, pid))    
            for modality in MODALITIES:
                if f'{modality}_L_image_data.npy' in ct_list:
                    l_ct_path = os.path.join(ori_root, pid, f'{modality}_L_image_data.npy')
                    r_ct_path = os.path.join(ori_root, pid, f'{modality}_R_image_data.npy')
                    l_img = deterministic_transform({'image': l_ct_path})['image']
                    r_img = deterministic_transform({'image': r_ct_path})['image']
                    np.save(os.path.join(save_path, pid, f'{modality}_L_image_data.npy'), l_img)
                    np.save(os.path.join(save_path, pid, f'{modality}_R_image_data.npy'), r_img)

        except Exception as e:
            print(f"Error processing PID {pid} in {dataset}: {e}")

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from PIL import Image
import json
from itertools import chain
import numpy as np

args = parser.parse_args()
args.resize_3d = 140
args.resize_slices = 32

deterministic_preprocess_transform_list = [
    LoadImaged(keys=["image"], reader='numpyreader'),
    EnsureChannelFirstd(keys=["image"]),
    Orientationd(keys=["image"], axcodes="RAS"),
    Resized(keys="image", spatial_size=[args.resize_3d, args.resize_3d, args.resize_slices]),
    OptimizedWindowLevelWidthd(keys=["image"], window_level=50, window_width=500),
]
deterministic_transform = Compose(deterministic_preprocess_transform_list)

datasets = ["XM_ALL", "LY_ALL", "ZY_ALL", "RJ_ALL", "SD_ALL", "TCIA_ALL"]
save_root = fr"/cpfs01/projects-SSD/cfff-bb5d866c17c2_SSD/public/RenalCLIP/nii_npy_aligned_preprocessed"
save_path = os.path.join(save_root, "one_kidney_tc_v3")

for dataset in datasets:
    print(f"Processing dataset: {dataset}")

    ori_root = fr"/cpfs01/projects-HDD/cfff-bb5d866c17c2_HDD/public/ExternalValid/{dataset}/crop_img-TC_v3"
    excel_path = "/cpfs01/projects-SSD/cfff-bb5d866c17c2_SSD/public/RenalCLIP/text_files/RenalCLIP_4.2.xlsx"
    df = pd.read_excel(excel_path, sheet_name=dataset.split("_")[0], dtype={"医技号": str})
    pid_side_map = dict(zip(df["医技号"], df["tumor_side"]))

    patient_list = os.listdir(ori_root)

    modalities = ["N", "A", "V", "D"]

    for pid in tqdm(patient_list):
        try:
            tumor_side = pid_side_map.get(pid)
            if not tumor_side:
                print(f"PID {pid} not found in tumor_side mapping.")
                continue

            save_dir = os.path.join(save_path, pid)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)

            ct_list = os.listdir(os.path.join(ori_root, pid))

            image_saved = False
            for modality in modalities:
                ct_filename = f'{modality}_{tumor_side}_image_data.npy'
                if ct_filename in ct_list:
                    ct_path = os.path.join(ori_root, pid, ct_filename)
                    img = deterministic_transform({'image': ct_path})['image']
                    np.save(os.path.join(save_dir, ct_filename), img)
                    print(f"Processed and saved {ct_filename} for PID {pid}.")
                    image_saved = True

            if not image_saved:
                print(f"No valid modality found for PID {pid}.")

        except Exception as e:
            print(f"Error processing PID {pid}: {e}")